# Group Project

Grzegorz Malisz & Tiemon Steeghs

[GitHub Repository](https://github.com/grzgm/deth-group-project)

## Context

We are focusing on creating a model that would ease and enhance the process of learning from MOOCs [Massive Open Online Courses]. The main goal of the model is to find the best learning path for user, based on user's current profile and his goal of finishing certain course. The path established by the agent should minimize the dropout possibility, learning curve and maximise the speed of learning. For our project we have picked the domain of IT MOOCs, but our solution can be extended to other topics.

## IT MOOC

For the development purpose we have used this structure of MOOC:

<!--!!!TODO
1. Statistical Methods in Number Theory for Beginners
   - Arithmetic
   - Statistics
2. Algebraic Statistics
   - Algebra
   - Statistics
3. Basic Arithmetic
   - Arithmetic
   - Geometry
   - Algebra
   - Calculus
4. Elementary Algebra
   - Arithmetic
   - Geometry
   - Algebra
5. Geometry:
   - Arithmetic
   - Geometry
!!!TODO-->

## MDP Formal Definition

### States

Skill levels in various domains: $(s_1, s_2, \ldots, s_N)$

Defined IT skills:

<!--!!!TODO
1. Arithmetic
2. Geometry
3. Algebra
4. Calculus

Descriptive visualisation of the state: $(Arithmetic, Geometry, Algebra, Calculus, Statistics, Discrete Mathematics, Logic, Mathematical Analysis)$

Later in the paper we will use the two notation, which ease understanding and writing while serving different purposes. For example, to refer to the Arithmetic skill level we will use $s_1$, as well as $s_{Arithmetic}$ notation.
!!!TODO-->

### Actions
Actions are defined by taking specific courses, each course has an associated upskilling vector $u_i$ and requirement vector $r_i$. Note that if value of the certain skill in the vector is 0, it is not denoted for the sake of clarity.

<!--!!!TODO
1. Statistical Methods in Number Theory for Beginners
   - Requirements vector:
     - $r_{Arithmetic} = 1$
     - $r_{Statistics} = 1$
   - Upskill vector:
     - $u_{Arithmetic} = 2$
     - $u_{Statistics} = 2$
2. Algebraic Statistics
   - Requirements vector:
     - $r_{Algebra} = 2$
     - $r_{Statistics} = 3$
   - Upskill vector:
     - $u_{Algebra} = 4$
     - $u_{Statistics} = 4$
3. Basic Arithmetic
   - Requirements vector:
     - None
   - Upskill vector:
     - $u_{Arithmetic} = 3$
     - $u_{Geometry} = 1$
     - $u_{Algebra} = 1$
     - $u_{Calculus} = 1$
4. Elementary Algebra
   - Requirements vector:
     - None
   - Upskill vector:
     - $u_{Arithmetic} = 1$
     - $u_{Geometry} = 1$
     - $u_{Algebra} = 2$
5. Geometry:
   - Requirements vector:
     - $r_{Arithmetic} = 1$
   - Upskill vector:
     - $u_{Arithmetic} = 2$
     - $u_{Geometry} = 2$
!!!TODO-->

### Transition Probabilities

Probability of passing a course is proportional to the dot product of the student's skill level and the course's requirement vector. 
- If the student passes the course, their skills are updated by $s + \alpha \cdot u_i \cdot x$ where $x$ is a random soft mask and $\alpha > \beta$.
- If the student fails, they might still get a slight skill improvement $s + \beta \cdot u_i \cdot x$ where $\beta < \alpha$

### Terminal State Probability

The transition probability for passing a course can be represented as:  
$$P(s, s^ \prime, a_i) = \frac{1}{1+ \exp(-\gamma r_i \cdot s)} $$

where $a_i$ is the action of taking course with requirement $r_i$ and course learning outcomes $r_i$  

The state transition for passing the course:  
$$ s^ \prime = s + \alpha \cdot u_i \odot x $$

And for failing:  
$$ s^ \prime = s + \beta \cdot u_i \odot x $$

where $x$ is a mask that attenuates entries of $m$ by element wise multiplication $\odot$ . Each student has its own $x$ such that each student has its own learning abilities.

The terminal state probability involves the student reaching the desired minimal skill level in all domains, leading to a reward of +1.

### Rewards

<!--!!!TODO
The rewards for the agent will be based on a few factors:
- Passing or failing a course
- How long the agent is learning (cost of living)
- Achieving the requested goal

Passing every course costs -1, and reaching the end state grants a reward of +1. To incentivize the agent to find the most efficient route a cost of living is taking into account. We made the decision to not implement this through states (as a budget for example) but through reward logic. Implementing this feature through states will lead to a lot more different states to calculate which is something we do not want, especially when the functionality is the same.
!!!TODO-->

## Implementation

### Mdp Class



Python `Mdp` Class is used mainly for storage of the States, Actions, Transition Probabilities and Rewards. In addition, it encapsulates logic of determining next State after Agents Action, for the Q-Learning and Monte-Carlo algorithms, helper methods for determining the available Actions for Agent and guard method for inspecting Transition Probabilities, so that they add up to 1 or 0 in case of the terminal state.

The `Mdp` Class is also taking advantage of the `EnvironmentBuilder` Class to build States, Actions, Transition Probabilities and Rewards from the given input of MOOC structure and formulas written above.

The MDP is devised to work with any type of the States and Actions. It is done by assigning to each given State or Action unique Id, which is used by the MDP for all operations. This allows to store the States and Actions in the NumPy Array, to save the computation resources and allow for great extensibility. In the implementation it is done via usage of the Python Lists, which store all the Possible States and Actions and therefore assign to each unique index.

### EnvironmentBuilder Class

<!--!!!TODO
Description of EnvironmentBuilder
!!!TODO-->

### Solver Class

`Solver Class` is used to structure and make accessible algorithms for finding the Optimal Policy. It works based on the data stored in the `Mdp` Class and consists of implementation of Dynamic Programming, Q-Learning and Monte-Carlo. Q-Learning and Monte-Carlo are implemented with two operation modes: based on fixed amount of episodes or until Policy has Converged.

The `Solver Class` is also responsible for generating the plots of the Action Value Function and Episode Returns. Each plot has visible Algorithm and Parameters used to obtain data, which allows for easy reruns of Algorithms and straightforward testing Algorithm performance.

`Solver Class` is mainly used with combination of three steps:
1. Resetting Solver to the initial values,
2. Running selected Algorithm with given values,
3. Displaying Plots.

## Algorithms

For the purpose of solving the problem we have implemented 3 different algorithms:

- Dynamic Programming,
- Q-Learning,
- Monte-Carlo.

Algorithms can be categorized based on their knowledge of transition probability. Dynamic Programming uses knowledge of all transition probabilities, where Q-Learning and Monte-Carlo work based on the data provided from mdp (states, actions, rewards).

In addition, to standard implementation of Q-Learning and Monte-Carlo with the Epsilon exploration, the guards for starting stage of algorithms were introduced, as if all Actions in the current state have Action Value of 0, the Agent chooses random Action instead of always first one.
`not np.max(self.action_value_array[previous_state, :])`

### Monte-Carlo

It is controlled by the variable `monte_carlo_enabled`. During the episode of the Agent, a history of choices is recorded, and after the episode finishes the Action Value Function is evaluated, by analysing the Agent path from the end to the beginning. Algorithm inspects whether maximal difference between old and new value is smaller than threshold `theta`, which indicates that the optimal Action Value Function for given policy has been found. Process stops repeating when the Policy has Converged. Monte-Carlo Evaluation formula:

$$
Q({s}^m_t, {a}^m_t) \gets Q({s}^m_t, {a}^m_t) + \alpha ({g}^m_t - Q({s}^m_t, {a}^m_t))
$$

### Q-Learning

It is controlled by the variable `q_learning_enabled` and evaluates the Action Value Function every step of the agent. Algorithm inspects whether maximal difference between old and new value is smaller than threshold `theta`, which indicates that the optimal Action Value Function for given policy has been found. Process stops repeating when the Policy has Converged. Q-Learning Evaluation formula:

<!--!!!TODO $$
action_value_array[previous_state, action] = (1 - alpha_q_learning) * action_value_array[
                previous_state, action] + alpha_q_learning * (reward + gamma * max(action_value_array[new_state, :]))
$$ !!!TODO-->

### Dynamic Programming

It is controlled by the variable `dynamic_programming_enabled` and does the Policy Evaluation and Policy Improvement to compute the Action Value Function based on the Greedy Policy with respect to Action Value Function, by implementation of the Bellman Equation:

$$
q_{\pi}(s,a)=\sum_{^{s^ \prime \in S}_{r \in R}} p(s^ \prime, r | s, a)[r + \gamma \sum_{a^ \prime \in A(s^ \prime)} \pi(a^ \prime | s^ \prime)q_{\pi}(s^ \prime | a^ \prime)]
$$

The $\sum_{a^ \prime \in A(s^ \prime)} \pi(a^ \prime | s^ \prime)q_{\pi}(s^ \prime | a^ \prime)$ inside the code is solved by usage of `action_value_array[new_state, np.argmax(action_value_array[new_state, :])]`, as the Greedy Policy chooses only one action, not any other, which means that in probabilities given by $\pi(a^ \prime | s^ \prime)$ they compose only of 0 and one 1, thus resulting in multiplying most of the values from the Action Value Function by 0. In order to omit unnecessary computation only the value of Action Value Function for the Action with probability of 1 is present.

#### Policy Evaluation

After each sweep through the Action Value Function the algorithm inspects whether maximal difference between old and new value is smaller than threshold `theta`, which indicates that the optimal Action Value Function for given policy has been found. Process stops repeating when the Policy has Converged.

#### Policy Improvement / Policy Extraction

Policy Improvement is done somewhat automatically, as the Policy is not stored, but is always calculated with the usage of `argmax()` function, which always returns Optimal Greedy Policy with respect to current Action Value Function.

## Conclusion

The model is incorporating the test data, and the agent is learning the optimal path to achieve the goal, but the model need further testing on more diverse scenarios. The ground basics for the project has been laid out. The concept of this Report can be extended to compensate possible users with not just the optimal learning path for their goal, but also it could balance between user goal and the most optimal skills for the current market. This approach would mean sacrificing user expectations, for his own good and possible success.

## Addtional Notes

# Redoing the same courses